In [2]:
import math
import networkx as nx
import matplotlib.pyplot as plt

class Node:
    def __init__(self, id, latitude, longitude, demand):
        self.id = id
        self.latitude = latitude
        self.longitude = longitude
        self.demand = demand

    def calculate_distance(self, node):
        return 100 * math.sqrt(
            math.pow(node.longitude - self.longitude, 2) +
            math.pow(node.latitude - self.latitude, 2)
        )
  
class Vehicle:
    def __init__(self, vehicle, maximum_capacity, cost):
        self.vehicle = vehicle
        self.maximum_capacity = maximum_capacity
        self.cost = cost

class Path:
    def __init__(self): 
        self.nodes = []

    def add_node(self, node):
        self.nodes.append(node)

    def calculate_total_distance(self):
        total_distance = 0
        for i in range(len(self.nodes) - 1):
            total_distance += self.nodes[i].calculate_distance(self.nodes[i+1])

        return total_distance
  
    def print_path(self):
        for index, node in enumerate(self.nodes):
            if index != len(self.nodes) - 1:
                print(node.id, end=' -> ')
            else:
                print(node.id)
                print()

    def print_output(self):
        pass

    def plot_path(self, figsize=(14, 8)):
        """
            Visualize the path
            Placement of label is not consistent 
        """
        G = nx.DiGraph()
        edges = []
        for index in range(len(self.nodes) - 1):
            edges.append(
            (
                self.nodes[index].id, 
                self.nodes[index+1].id, 
                {'weight': round(self.nodes[index].calculate_distance(self.nodes[index+1]), 4)} # weight between 2 nodes
            )
            )

        G.add_edges_from(edges)
        plt.figure(figsize=figsize)
        pos = nx.circular_layout(G)
        nx.draw(G, with_labels=True)
        edge_weight = nx.get_edge_attributes(G,'weight')
        nx.draw_networkx_edge_labels(G, pos, edge_labels = edge_weight)
        plt.show()

In [3]:
import numpy as np

def calculate_customer_heuristics():
    """
        Calculate heuristics for the customer allocation's state transition rules
        Calculates ratio of capacity to fixed cost
        Caclulate shortest distance among all edges between the customers of same vehicle type
    """
    
    pass 

def allocate_customer():
    """
        Allocate customer to each type of vehicle
        First customer is assigned randomly
        Remaining customer is assigned based on the state transition rule
        State rule: (Pheromone x heuristics)
    """
    pass 

def update_global_pheromone_cust():
    """
        Update pheromone based on the best solution so far and best solution in this iteration
    """
    pass 

def update_local_pheromone_cust():
    """
        Update local pheromone matrix for customer allocation stage
    """
    pass 

def calculate_route_heuristics(
    pheromone, # pheromone level for that route
    depot_node,
    source_node,
    dst_node,
    beta, # coefficient for n
):
    """
        Calculate heuristics for route construction
    """
    n = source_node.calculate_distance(depot_node) + depot_node.calculate_distance(dst_node) - source_node.calculate_distance(dst_node)
    
    return pheromone * (math.pow(n, beta))


def decide_next_node(
    vertices, # all vertices available for the type of fleet
    pheromone_matrix,
    visited_nodes,
    current_node,
):
    """
        Decide next node based on the probability calculated 
    """
    BETA = 2 

    heuristic_nodes = []

    for next_node in range(len(vertices)):
        if next_node == 0 or visited_nodes[next_node]:
            # if either visited nodes or depot node
            heuristic_nodes.append(0) # make the model not selecting visited node

        else:
            route_heuristic = calculate_route_heuristics(
                pheromone_matrix[current_node][next_node], # pheromone for that edge
                vertices[0], # depot
                vertices[current_node], # node i
                vertices[next_node], # node j
                BETA
            )

            heuristic_nodes.append(route_heuristic)

    heuristic_nodes = np.array(heuristic_nodes)
    heuristic_nodes = heuristic_nodes / heuristic_nodes.sum()
    next_node = np.argmax(heuristic_nodes)

    return next_node

def update_global_pheromone_route(
    pheromone, # existing pheromone
    global_best_distance, 
    local_best_distance,
    third_distance,
    pheromone_coefficient 
):
    """
        Update global pheromone matrix for route construction
    """
    delta_pheromone = ((third_distance - global_best_distance) + (third_distance - local_best_distance)) / third_distance
    new_pheromone = ((1 - pheromone_coefficient) * (pheromone)) + (pheromone_coefficient * delta_pheromone)
    
    return new_pheromone

def update_local_pheromone_route(
    pheromone,
    evaporation_coefficient, # user-defined parameter
    initial_values # user-defined parameter
):
    """
        Update local pheromone for route construction (evaporation)
    """
    pheromone_matrix = pheromone + (evaporation_coefficient * initial_values)

    return pheromone_matrix

def aco():
    pass 
